## Imports

In [ ]:
# Standard library imports
import math
from math import lgamma

# Third party imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import poisson
from scipy.stats import beta
from scipy.stats import chi2

## Load Data

In [ ]:
# Load the Two Currencies CSV
df = pd.read_csv("two_currencies.csv")

In [ ]:
# Load the Student Performance CSV
df = pd.read_csv("student_performance.csv")

## Question 2

---

**Expected Portfolio Profit**

$
E[\Pi] = b_A E[P_A] + b_B E[P_B]
$

**Constraint**

$
b_A + b_B = 20{,}000
$

**Portfolio Variance**

$
\operatorname{Var}(\Pi) = b_A^2 \, \operatorname{Var}(P_A) + b_B^2 \, \operatorname{Var}(P_B) + 2 b_A b_B \, \operatorname{Cov}(P_A, P_B)
$

**Optimal Allocation (Minimum Variance)**

$
b_A^* = 20{,}000 \cdot \frac{\operatorname{Var}(P_B) - \operatorname{Cov}(P_A, P_B)}{\operatorname{Var}(P_A) + \operatorname{Var}(P_B) - 2 \operatorname{Cov}(P_A, P_B)}
$

$
b_B^* = 20{,}000 - b_A^*
$

In [ ]:
# Extract profits (these are for $10,000 investment)
PA = df["profit_A"].values
PB = df["profit_B"].values

# Part (a)
# Maximize expected profit
mean_PA = np.mean(PA)
mean_PB = np.mean(PB)

print("Mean profit A (per $10k):", mean_PA)
print("Mean profit B (per $10k):", mean_PB)

if mean_PA > mean_PB:
    print("Max profit: invest all in A")
elif mean_PA < mean_PB:
    print("Max profit: invest all in B")
else:
    print("Any split works, means are equal.")

# Part (b)
# Minimize variance (risk)
var_A = np.var(PA, ddof=1)
var_B = np.var(PB, ddof=1)
cov_AB = np.cov(PA, PB, ddof=1)[0, 1]

bA_opt = 20000 * (var_B - cov_AB) / (var_A + var_B - 2 * cov_AB)
bB_opt = 20000 - bA_opt

print("Optimal b_A for minimum variance:", bA_opt)
print("Optimal b_B for minimum variance:", bB_opt)

# Grid search to verify
amounts = np.linspace(0, 20000, 201)
variances = []

for bA in amounts:
    bB = 20000 - bA
    profits_scaled_A = (bA / 10000) * PA
    profits_scaled_B = (bB / 10000) * PB
    portfolio_profits = profits_scaled_A + profits_scaled_B
    variances.append(np.var(portfolio_profits, ddof=1))

min_idx = np.argmin(variances)
print(
    "Grid search min variance allocation:", amounts[min_idx], 20000 - amounts[min_idx]
)

## Question 3

---

**Hypotheses**

$
H_0 : \mu_{\text{female}} = \mu_{\text{male}}
$

$
H_A : \mu_{\text{female}} \neq \mu_{\text{male}}
$

**Correlation (Pearson)**

$
r_{XY} = \frac{\operatorname{Cov}(X,Y)}{\sigma_X \sigma_Y}
$

**Sample covariance**

$
\operatorname{Cov}(X,Y) = \frac{1}{n-1} \sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})
$

**Sample standard deviation**

$
\sigma_X = \sqrt{\frac{1}{n-1} \sum_{i=1}^n (x_i - \bar{x})^2}
$


**Two-sample Welch’s t-test statistic**

$
t = \frac{\bar{x}_1 - \bar{x}_2}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$

**Degrees of freedom (Welch–Satterthwaite equation)**

$
df = \frac{\left(\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}\right)^2}{\frac{\left(\tfrac{s_1^2}{n_1}\right)^2}{n_1 - 1} + \frac{\left(\tfrac{s_2^2}{n_2}\right)^2}{n_2 - 1}}
$

In [ ]:
# Encode sex: female=1, male=0
df["sex_code"] = df["sex"].map({"F": 1, "M": 0})

# Part (a)
# Compute correlation
r = df["sex_code"].corr(df["G1"])
print("Correlation between gender and G1:", r)

# Part (b)
# Split into two groups
females = df[df["sex"] == "F"]["G1"]
males = df[df["sex"] == "M"]["G1"]

# Welch’s df
n1 = len(females)
n2 = len(males)
s1 = np.var(females, ddof=1)
s2 = np.var(males, ddof=1)

dof = ((s1 / n1 + s2 / n2) ** 2) / (
    ((s1 / n1) ** 2) / (n1 - 1) + ((s2 / n2) ** 2) / (n2 - 1)
)
print("Degrees of freedom:", dof)

# Welch’s t-test
t_stat, p_val = stats.ttest_ind(females, males, equal_var=False)

print("t-statistic:", t_stat)
print("p-value:", p_val)

if p_val < 0.05:
    print("Reject H0: Significant difference in average G1.")
else:
    print("Fail to reject H0: No significant difference in average G1.")

## Question 4

---

**Hypotheses**

$
H_0 : \mu_{G1} = \mu_{G2}
$

$
H_A : \mu_{G1} \neq \mu_{G2}
$

**Paired t-test statistic**

$
t = \frac{\bar{d}}{s_d / \sqrt{n}}
$

**Where**

$
d_i = G1_i - G2_i
$

$
\bar{d} = \frac{1}{n} \sum_{i=1}^n d_i
$

$
s_d^2 = \frac{1}{n-1} \sum_{i=1}^n (d_i - \bar{d})^2
$

In [ ]:
# Difference between grades
diff = df["G1"] - df["G2"]
# Paired t-test on G1 and G2
t_stat, p_val = stats.ttest_rel(df["G1"], df["G2"])

print("t-statistic:", t_stat)
print("p-value:", p_val)

# One-sample t-test on difference vs zero
t_stat, p_val = stats.ttest_1samp(diff, 0)

print("t-statistic:", t_stat)
print("p-value:", p_val)

if p_val < 0.05:
    print("Reject H0: Significant difference between G1 and G2.")
else:
    print("Fail to reject H0: No significant difference.")

## Question 5

---

**Parameter and estimator**

$
\mu = E[G3], \quad \hat{\mu} = \bar{X} = \frac{1}{n}\sum_{i=1}^n X_i
$

**Empirical distribution (nonparametric bootstrap)**

$
\hat{F}_n = \frac{1}{n}\sum_{i=1}^n \delta_{X_i}
$

**Bootstrap resamples and replicates (nonparametric)**

$
X^{*b}_1,\dots,X^{*b}_n \overset{i.i.d.}{\sim} \hat{F}_n,\quad 
\hat{\mu}^{*b} = \frac{1}{n}\sum_{i=1}^n X^{*b}_i,\quad b=1,\dots,B
$

**Percentile confidence interval (level $0.9$)**

$
\text{CI}_{0.9}^{\text{perc}} = \left[\, Q_{0.05}\big(\{\hat{\mu}^{*b}\}_{b=1}^B\big),\; Q_{0.95}\big(\{\hat{\mu}^{*b}\}_{b=1}^B\big) \,\right]
$

**Bootstrap standard error (optional)**

$
\widehat{SE}_{\text{boot}}(\hat{\mu}) = \sqrt{\frac{1}{B-1}\sum_{b=1}^B \left(\hat{\mu}^{*b}-\overline{\hat{\mu}^{*}}\right)^2},\quad 
\overline{\hat{\mu}^{*}}=\frac{1}{B}\sum_{b=1}^B \hat{\mu}^{*b}
$

**Parametric bootstrap (example: normal fit)**

$
\hat{\mu}=\bar{X},\quad \hat{\sigma}^2 = s^2=\frac{1}{n-1}\sum_{i=1}^n (X_i-\bar{X})^2,
$

$
X^{*b}_i \overset{i.i.d.}{\sim} \mathcal{N}(\hat{\mu},\hat{\sigma}^2),\quad 
\hat{\mu}^{*b}=\frac{1}{n}\sum_{i=1}^n X^{*b}_i,\quad 
\text{CI}_{0.9}^{\text{perc}}=\left[Q_{0.05},\,Q_{0.95}\right]
$

In [ ]:
g3 = df["G3"].values
B = 10000
boot_means = []

# Generate bootstrap samples and compute means
np.random.seed(0)
for _ in range(B):
    sample = np.random.choice(g3, size=len(g3), replace=True)
    boot_means.append(sample.mean())

# Calculate 90% confidence interval using percentiles
lower = np.percentile(boot_means, 5)
upper = np.percentile(boot_means, 95)

print(f"0.9 Bootstrap CI for mean G3: ({lower:.2f}, {upper:.2f})")

## Question 6

---

**Sample mean**

$
\bar{X} = \frac{1}{n} \sum_{i=1}^n X_i
$

**Sample standard deviation**

$
s = \sqrt{\frac{1}{n-1} \sum_{i=1}^n (X_i - \bar{X})^2}
$

**Central Limit Theorem (approximate sampling distribution)**

$
\bar{X} \;\approx\; \mathcal{N}\!\left(\mu,\; \frac{\sigma^2}{n}\right)
$

**Standard error (estimated)**

$
SE = \frac{s}{\sqrt{n}}
$

**Critical value for 90% CI (two-sided)**

$
z_{0.95} = 1.645
$

**Margin of error**

$
ME = z_{0.95} \cdot SE
$

**Confidence interval (CLT-based)**

$
\text{CI}_{0.9} = \Big[\, \bar{X} - ME,\; \bar{X} + ME \,\Big]
$

In [ ]:
mean_g3 = np.mean(g3)
std_g3 = np.std(g3, ddof=1)
n = len(g3)
z_crit = 1.645

# Margin of error
margin = z_crit * (std_g3 / math.sqrt(n))

# Lower and upper bounds of 90% confidence interval
lower_clt = mean_g3 - margin
upper_clt = mean_g3 + margin

print(f"0.9 CLT CI for mean G3: ({lower_clt:.2f}, {upper_clt:.2f})")

## Question 7

---

### Compare bootstrap vs CLT CIs

* Both methods provide very similar 0.9 confidence intervals:  
  * Bootstrap CI: (11.69, 12.11)  
  * CLT CI: (11.70, 12.11)  
* This similarity indicates that the sample size is large enough for the CLT to hold, making its normality assumption appropriate.  
* Bootstrap does not rely on parametric assumptions and uses resampling to estimate the CI, making it more flexible.  
* The slight difference in lower bound is negligible and due to bootstrap capturing sample variability directly.  
* Overall, the consistent results show both methods are reliable for this data.

## Question 9

---

**Likelihood (Poisson, iid)**

$
L(\lambda; x_1,\dots,x_n) \;=\; \prod_{i=1}^n \frac{e^{-\lambda}\lambda^{x_i}}{x_i!}
$

**Log-likelihood**

$
\ell(\lambda)=\log L(\lambda)= -n\lambda + \left(\sum_{i=1}^n x_i\right)\log\lambda - \sum_{i=1}^n \log(x_i!)
$

**Maximum Likelihood Estimator (MLE)**

$
\hat{\lambda} = \bar{X} = \frac{1}{n}\sum_{i=1}^n x_i
$

**Log-likelihood at $\hat\lambda$**

$
\ell(\hat{\lambda}) = -n\hat{\lambda} + \left(\sum_{i=1}^n x_i\right)\log\hat{\lambda} - \sum_{i=1}^n \log(x_i!)
$

In [ ]:
# Extract absences column, drop missing
absences = df["absences"]

# Summary stats
n = len(absences)
sum_x = absences.sum()
x_bar = absences.mean()

print("Number of students (n):", n)
print("Total absences (Σ xi):", sum_x)
print("Sample mean (x̄):", x_bar)

# Poisson MLE (λ̂ = sample mean)
lambda_hat = x_bar
print("Poisson MLE λ̂ =", lambda_hat)

## Question 10

---

**Exponential PDF:**

$
f(t;\lambda) = \lambda e^{-\lambda t}, \quad t \ge 0
$

**Interval probability (discretization):**

$
P(X = x) \approx \int_{x-0.5}^{x+0.5} \lambda e^{-\lambda t} \, dt
= e^{-\lambda (x-0.5)} \big(1 - e^{-\lambda}\big)
$

**Log-likelihood:**

$
\ell(\lambda) = \sum_{i=1}^n \log P(X=x_i) 
= -\lambda \sum_{i=1}^n (x_i - 0.5) + n \log\big(1 - e^{-\lambda}\big)
$

**Score equation:**

$
\frac{\partial}{\partial \lambda} \ell(\lambda) = -S + \frac{n e^{-\lambda}}{1 - e^{-\lambda}} = 0
$

**MLE solution:**

$
\hat{\lambda} = \log\!\left(1 + \frac{n}{S}\right), \quad S = \sum_{i=1}^n (x_i - 0.5)
$

**Log-likelihood at MLE:**

$
\ell(\hat{\lambda}) = -\hat{\lambda} S + n \log\big(1 - e^{-\hat{\lambda}}\big)
$

A Poisson model gives probabilities for exact integer counts, while an exponential model gives continuous densities. Since densities aren't directly comparable to probabilities, we convert the exponential density into interval probabilities around each integer (e.g., [x−0.5, x+0.5)) to fairly compare it with the Poisson probabilities.

In [ ]:
# Exponential MLE
S = sum_x - 0.5 * n
lambda_hat_exp = math.log(1.0 + n / S)

loglik_exp = -lambda_hat_exp * S + n * math.log(1 - math.exp(-lambda_hat_exp))

print("Exponential model (interval method):")
print("  λ̂ =", lambda_hat_exp)
print("  log-likelihood =", loglik_exp)

# Log-likelihood at λ̂ (Question 9)
log_factorials = sum(lgamma(int(x) + 1) for x in absences)
loglik = -n * lambda_hat + sum_x * math.log(lambda_hat) - log_factorials
print("Poisson Log-likelihood at λ̂:", loglik)

# Compare Poisson vs Exponential
if loglik > loglik_exp:
    print("Poisson provides the better fit (higher likelihood).")
elif loglik < loglik_exp:
    print("Exponential provides the better fit (higher likelihood).")
else:
    print("Both models fit equally well (same likelihood).")

## Question 11

---

**Empirical probability mass function (empirical PDF):**

$
\hat{p}(x) = \frac{\#\{i : X_i = x\}}{n}, \quad x \in \mathbb{Z}_{\ge 0}
$

**Poisson PMF with MLE $\hat{\lambda}$:**

$
p(x; \hat{\lambda}) = \frac{e^{-\hat{\lambda}} \hat{\lambda}^x}{x!}, \quad x \in \mathbb{Z}_{\ge 0}
$

In [ ]:
# Empirical PDF
values, counts = np.unique(absences, return_counts=True)
empirical_pdf = counts / n

# Poisson pmf using λ̂
x_vals = np.arange(0, max(absences) + 1)
poisson_pmf = poisson.pmf(x_vals, mu=lambda_hat)

# Plot
plt.figure(figsize=(8, 6))
plt.bar(values, empirical_pdf, width=0.6, alpha=0.6, label="Empirical PDF")
plt.plot(x_vals, poisson_pmf, "ro-", label=f"Poisson(λ={lambda_hat:.2f}) PMF")
plt.xlabel("Absences")
plt.ylabel("Probability")
plt.title("Empirical PDF vs Poisson likelihood function")
plt.legend()
plt.show()

## Question 12

---

**Empirical CDF:**

$
\hat{F}(x) = \frac{1}{n} \sum_{i=1}^n \mathbf{1}\{X_i \leq x\}, \quad x \in \mathbb{R}
$

**Poisson CDF with MLE $\hat{\lambda}$:**

$
F(x; \hat{\lambda}) = \Pr(X \leq x) = \sum_{k=0}^{\lfloor x \rfloor} \frac{e^{-\hat{\lambda}} \hat{\lambda}^k}{k!}, \quad x \in \mathbb{Z}_{\ge 0}
$

In [ ]:
# Empirical CDF
sorted_abs = np.sort(absences)
empirical_cdf = np.arange(1, n + 1) / n

# Poisson CDF
poisson_cdf = poisson.cdf(x_vals, mu=lambda_hat)

# Plot
plt.figure(figsize=(8, 6))
plt.step(sorted_abs, empirical_cdf, where="post", label="Empirical CDF")
plt.plot(x_vals, poisson_cdf, "r-", label=f"Poisson(λ={lambda_hat:.2f}) CDF")
plt.xlabel("Absences")
plt.ylabel("Cumulative probability")
plt.title("Empirical CDF vs Poisson CDF")
plt.legend()
plt.show()

## Question 13

---

### Does the Distribution Fit?

* **PDF (Q11):**

  * Empirical PDF has a large spike at **0 absences**, much higher than Poisson predicts.
  * Poisson fits moderately well for values around **2–6 absences**.
  * Poisson decays too fast → underestimates the **heavy tail** (students with many absences).

* **CDF (Q12):**

  * Poisson CDF rises faster than empirical → puts too much probability on **small values**.
  * Empirical CDF is more gradual, reflecting a **wider spread** in the data.

**Conclusion**: The Poisson distribution does not fully capture the absences data. It misses the excess zeros and longer tail, suggesting overdispersion.

## Question 14

---

**Hypotheses**

* $H_0 :$ Fjob and famsize are independent.
* $H_A :$ Fjob and famsize are dependent.

**Expected counts under independence**

$
E_{ij} = \frac{(\text{row total}_i)(\text{column total}_j)}{n}
$

**Chi-square statistic**

$
\chi^2 = \sum_{i=1}^{r} \sum_{j=1}^{c} \frac{(O_{ij} - E_{ij})^2}{E_{ij}}
$

**Degrees of freedom**

$
df = (r - 1)(c - 1)
$

**p-value (Chi-square distribution)**

$
p = 1 - F_{\chi^2}(\chi^2; df)
$

In [ ]:
# Build observed contingency table
ct = pd.crosstab(df["Fjob"], df["famsize"])
print("Observed contingency table:")
print(ct)

# Manual calculation (explicit)
observed = ct.values.astype(float)
row_totals = observed.sum(axis=1, keepdims=True)
col_totals = observed.sum(axis=0, keepdims=True)
n = observed.sum()
expected = (row_totals @ col_totals) / n

# check small expected counts
print("\nExpected counts (under independence):")
print(pd.DataFrame(expected, index=ct.index, columns=ct.columns))

# compute Pearson's X^2
chi2_stat = ((observed - expected) ** 2 / expected).sum()
r, c = observed.shape
dof = (r - 1) * (c - 1)
p_val = 1 - stats.chi2.cdf(chi2_stat, dof)

print("\nManual chi-square statistic:", chi2_stat)
print("Degrees of freedom:", dof)
print("p-value:", p_val)

# Using scipy convenience function (returns same statistic)
chi2_stat, p_val, dof, expected = stats.contingency.chi2_contingency(observed, correction=False)

print("\nscipy chi-square statistic:", chi2_stat)
print("scipy degrees of freedom:", dof)
print("scipy p-value:", p_val)

# expected returned by scipy should match expected
print("\nExpected counts (from scipy):")
print(pd.DataFrame(expected, index=ct.index, columns=ct.columns))

# Decision at alpha = 0.05
alpha = 0.05
if p_val < alpha:
    print("\nReject H0: Evidence of dependence between Fjob and famsize.")
else:
    print("\nFail to reject H0: No evidence of dependence between Fjob and famsize.")

## Question 15

---

**Sample correlation (Pearson’s r)**

$
r_{XY} = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{(n-1) s_X s_Y}
$

**Sample covariance (inside numerator)**

$
\text{Cov}(X,Y) = \frac{1}{n-1} \sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})
$

**Standard deviations**

$
s_X = \sqrt{\frac{1}{n-1} \sum_{i=1}^n (x_i - \bar{x})^2}, \quad 
s_Y = \sqrt{\frac{1}{n-1} \sum_{i=1}^n (y_i - \bar{y})^2}
$

In [ ]:
cols = ["famrel", "freetime", "goout", "Dalc", "Walc", "health", "G3"]

# Compute correlations with G3
corrs = df[cols].corr()["G3"].drop("G3")  # correlations with G3 only
print("Correlations with G3:")
print(corrs)

# Find strongest
strongest_var = corrs.abs().idxmax()
print(f"\nStrongest correlation: {strongest_var}, r = {corrs[strongest_var]:.3f}")

# Visualization 1: correlation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(df[cols].corr(), annot=True, cmap="coolwarm", center=0)
plt.title("Correlation matrix (G3 vs predictors)")
plt.show()

## Question 16

---

**Hypotheses**

$
H_0 : \rho \leq -0.2
$

$
H_A : \rho > -0.2
$

**Bootstrap resampling:**

For $b = 1, 2, \dots, B$:

$
r^{*(b)} = \text{corr}(X^*_b, Y^*_b)
$

where $X^*_b, Y^*_b$ are sampled with replacement from the original data.

**Bootstrap p-value:**

$
p = \frac{1}{B} \sum_{b=1}^B I\big(r^{*(b)} \leq -0.2\big)
$

where $I(\cdot)$ is an indicator function.

In [ ]:
# Extract variables
x = df["Walc"].values
y = df["G3"].values
n = len(x)

# Observed correlation
obs_corr = np.corrcoef(x, y)[0, 1]
print("Observed correlation:", obs_corr)

# Bootstrap
B = 10000
boot_corrs = []

np.random.seed(0)
for _ in range(B):
    # resample indices with replacement
    idx = np.random.choice(n, size=n, replace=True)
    x_resample = x[idx]
    y_resample = y[idx]
    boot_corrs.append(np.corrcoef(x_resample, y_resample)[0, 1])

boot_corrs = np.array(boot_corrs)

# p-value: proportion of bootstrap correlations ≤ -0.2
p_value = np.mean(boot_corrs <= -0.2)

print(f"Bootstrap p-value for H0: ρ <= -0.2 vs HA: ρ > -0.2 = {p_value:.4f}")

# Decision at alpha = 0.05
alpha = 0.05
if p_value < alpha:
    print("Reject H0: evidence that correlation is greater than -0.2")
else:
    print("Fail to reject H0: not enough evidence")

## Question 18

---

**Slope (β₁):**

$
\beta_1 = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2}
$

**Intercept (β₀):**

$
\beta_0 = \bar{y} - \beta_1 \bar{x}
$

**Regression line:**

$
\hat{y} = \beta_0 + \beta_1 x
$

In [ ]:
x = df["Dalc"].values
y = df["G3"].values

# Means
x_bar = np.mean(x)
y_bar = np.mean(y)

# Slope (beta1)
beta1 = np.sum((x - x_bar) * (y - y_bar)) / np.sum((x - x_bar) ** 2)

# Intercept (beta0)
beta0 = y_bar - beta1 * x_bar

print(f"Regression equation: G3 = {beta0:.3f} + {beta1:.3f} * Dalc")
print(f"Intercept: {beta0:.3f} (expected grade when Dalc=0)")
print(f"Slope: {beta1:.3f} (grade change per unit increase in Dalc)")

# Predictions for plotting
y_hat = beta0 + beta1 * x

# Scatterplot + regression line
plt.scatter(x, y, alpha=0.5, label="Data")
plt.plot(x, y_hat, color="red", label="Fitted line")
plt.xlabel("Dalc (workday alcohol consumption)")
plt.ylabel("G3 (final grade)")
plt.title("Simple Linear Regression: G3 vs Dalc")
plt.legend()
plt.show()

## Question 19

---

**Likelihood (Bernoulli trials):**

$
L(\theta) = \theta^k (1 - \theta)^{n-k}
$

**Bayes numerator (unnormalized posterior):**

$
\text{num}(\theta) = \pi(\theta) \cdot L(\theta)
$

**Posterior (normalized):**

$
p(\theta \mid \text{data}) = \frac{\pi(\theta) \cdot L(\theta)}{\int_0^1 \pi(\theta) \cdot L(\theta) \, d\theta}
$

**Log-likelihood (numerical stability):**

$
\log L(\theta) = k \log(\theta) + (n-k)\log(1-\theta)
$

(Then exponentiate after centering by `max`.)

**Theoretical posterior (Beta conjugacy):**

$
\theta \mid \text{data} \sim \text{Beta}(\alpha, \beta), \quad \alpha = k+1, \; \beta = n-k+1
$

**Estimates:**

* **MLE:**

$
\hat{\theta}_{MLE} = \frac{k}{n}
$

* **MAP:**

$
\hat{\theta}_{MAP} = \arg\max_\theta p(\theta \mid \text{data})
$

* **Posterior mean:**

$
E[\theta \mid \text{data}] = \frac{k+1}{n+2}
$

* **Posterior variance:**

$
\mathrm{Var}(\theta \mid \text{data}) = \frac{(k+1)(n-k+1)}{(n+2)^2 (n+3)}
$

In [ ]:
# Part (1)
# MAP vs MLE
df["higher_code"] = df["higher"].map({"yes": 1, "no": 0})
n = len(df["higher_code"])
k = df["higher_code"].sum()

theta = np.linspace(0.001, 0.999, 1000)
likelihood = theta**k * (1 - theta) ** (n - k)
posterior = likelihood / likelihood.sum()

theta_map = theta[np.argmax(posterior)]
theta_mle = k / n

print("MLE =", theta_mle)
print("MAP =", theta_map)

# Part (2)
# Prior and Likelihood
prior = np.ones_like(theta)
likelihood = theta**k * (1 - theta) ** (n - k)

# Part (3)
# Bayes numerator and posterior
numerator = prior * likelihood
posterior = numerator / numerator.sum()

# Part (4)
# Log-likelihood for stability
log_likelihood = k * np.log(theta) + (n - k) * np.log(1 - theta)
log_num = log_likelihood
posterior_log = np.exp(log_num - log_num.max())
posterior_log = posterior_log / posterior_log.sum()

# Part (5)
# Compare MAP, MLE, and posterior mean
theta_map = theta[np.argmax(posterior_log)]
theta_mle = k / n
posterior_mean = (k + 1) / (n + 2)
posterior_variance = ((k + 1) * (n - k + 1)) / ((n + 2)**2 * (n + 3))

print("MLE =", theta_mle)
print("MAP =", theta_map)
print("Posterior mean =", posterior_mean)
print("Posterior variance =", posterior_variance)

# Part (6)
# Theoretical posterior vs Simulation
alpha = k + 1
beta_param = n - k + 1
theory_pdf = beta.pdf(theta, alpha, beta_param)
posterior_log_density = posterior_log / (theta[1] - theta[0])

plt.plot(theta, posterior_log_density, label="Simulation posterior")
plt.plot(theta, theory_pdf, "--", label=f"Theoretical Beta({alpha},{beta_param})")
plt.legend()
plt.show()

## Question 21

---

**Hypotheses**

* $H_0:$ Rows and columns are independent.
* $H_A:$ Rows and columns are dependent.

**Expected counts under independence**

$
E_{ij} = \frac{(\text{row total}_i)(\text{column total}_j)}{n}
$

**Chi-square statistic**

$
\chi^2 = \sum_{i=1}^{R} \sum_{j=1}^{C} \frac{(O_{ij} - E_{ij})^2}{E_{ij}}
$

**Degrees of freedom**

$
df = (R - 1)(C - 1)
$

**Empirical p-value (simulation-based)**

$
\hat{p} = \frac{\text{\#}\{\chi^2_{\text{sim}} \geq \chi^2_{\text{obs}}\}}{\text{\# simulations}}
$

**Theoretical p-value (Chi-square distribution)**

$
p = 1 - F_{\chi^2}(\chi^2_{\text{obs}}; df)
$

In [ ]:
# Part (1): Empirical Chi-Squared Distribution
# Step 1: Choose dimensions
R, C = 3, 4
dof = (R - 1) * (C - 1)
print(f"Table dimensions: {R}x{C}, Degrees of freedom = {dof}")

# Step 2: Define null probability model (independence)
row_probs = np.array([0.3, 0.5, 0.2])
col_probs = np.array([0.2, 0.3, 0.25, 0.25])
P = np.outer(row_probs, col_probs)

# Step 3: Fix sample size
n = 200
print(f"Sample size = {n}")

# Step 4: Simulate many samples under null model
num_sims = 10000
chi2_values = []

np.random.seed(0)
for _ in range(num_sims):
    sample = np.random.multinomial(n, P.ravel()).reshape(R, C)
    O = sample
    row_totals = O.sum(axis=1, keepdims=True)
    col_totals = O.sum(axis=0, keepdims=True)
    E = row_totals @ col_totals / n
    chi2_stat = np.sum((O - E) ** 2 / E)
    chi2_values.append(chi2_stat)

chi2_values = np.array(chi2_values)

# Step 5 + 6: Plot histogram with theoretical chi-squared PDF
plt.hist(chi2_values, bins=40, density=True, alpha=0.6, label="Empirical")
x = np.linspace(0, np.max(chi2_values), 500)
plt.plot(x, chi2.pdf(x, dof), "r-", lw=2, label=f"Chi2 PDF (df={dof})")
plt.xlabel("Chi-squared statistic")
plt.ylabel("Density")
plt.title("Empirical vs Theoretical Chi-squared Distribution")
plt.legend()
plt.show()

# Step 7: Discussion prints
print("Part 1 Discussion")
print("The empirical histogram closely follows the theoretical Chi-squared distribution.")
print("With 10,000 simulations, the match is good. With fewer (e.g., 1,000), variability is higher.")
print("If n is too small, expected counts may fall below 5, causing deviations.")

# Part (2): Hypothesis Testing with Alternative Model
# Step 1: Define alternative probability model (dependence)
P_alt = P.copy()
boost = 0.02
for i in range(min(R, C)):
    P_alt[i, i] += boost
# renormalize
P_alt = P_alt / P_alt.sum()

# Step 2: Draw one sample from alternative model
np.random.seed(0)
S_alt = np.random.multinomial(n, P_alt.ravel()).reshape(R, C)
O_alt = S_alt

# Step 3: Compute chi-squared statistic against null expected counts
row_totals_alt = O_alt.sum(axis=1, keepdims=True)
col_totals_alt = O_alt.sum(axis=0, keepdims=True)
E_null = row_totals_alt @ col_totals_alt / n
chi2_obs = np.sum((O_alt - E_null) ** 2 / E_null)

# Step 4: Compute empirical p-value
p_value_emp = np.mean(chi2_values >= chi2_obs)

# Step 5: Decision at alpha = 0.1
alpha = 0.1
if p_value_emp < alpha:
    print("\nReject H0: Evidence of dependence between rows and columns.")
else:
    print("\nFail to reject H0: No evidence of dependence between rows and columns.")

# Step 6: Compare with theoretical p-value
p_value_theory = 1 - chi2.cdf(chi2_obs, dof)

print("\nPart 2 Results")
print("Observed chi-squared statistic:", chi2_obs)
print("Empirical p-value:", p_value_emp)
print("Theoretical p-value:", p_value_theory)

print("\nPart 2 Discussion")
print("The test rejected the null because dependence was introduced (boosted diagonal).")
print("The empirical p-value and theoretical p-value are close, but small differences occur due to simulation variability.")
print("The power of the test increases with larger sample size n, making dependence easier to detect.")